# Clean rent data #

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', None)

In [2]:
rent_all_columns = pd.read_csv('rent_data_all.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'rent_data_all.csv'

In [3]:
rent_all_columns.head(2)

,Unnamed: 0,bathrooms,bedrooms,city,country,currency,daysOnZillow,homeStatus,homeStatusForHDP,homeType,isFeatured,isNonOwnerOccupied,isPreforeclosureAuction,isPremierBuilder,isRentalWithBasePrice,isUnmappable,isZillowOwned,latitude,livingArea,longitude,price,priceForHDP,shouldHighlight,state,streetAddress,unit,zipcode,zpid,lotAreaUnit,lotAreaValue,rentZestimate,taxAssessedValue,zestimate,datePriceChanged,priceChange,priceReduction
0,0,1.0,1.0,Springfield,USA,USD,-1,FOR_RENT,FOR_RENT,APARTMENT,True,True,False,False,False,False,False,42.104000,617.0,-72.592384,1042.0,1042.0,False,MA,193 Worthington St #71DFDFD55,# 71dfdfd55,1103,2066685043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1.0,Springfield,USA,USD,-1,FOR_RENT,FOR_RENT,SINGLE_FAMILY,False,True,False,False,False,False,False,42.095623,1550.0,-72.496870,750.0,750.0,False,MA,42 Catalina Dr,NaN,1128,56205755,acres,0.579982,2400.0,212200.0,304700.0,NaN,NaN,NaN


### Renaming columns ###

In [4]:
rent_all_columns=rent_all_columns.rename(columns={'livingArea':'area', 'price':'rent', 'streetAddress':'street_address'})


### Adding a 0 prefix to all zipcodes ###

In [5]:
rent_all_columns['zipcode']=rent_all_columns['zipcode'].apply(lambda x: '{0:0>5}'.format(x))

In [6]:
rent_all_columns.head(1)

,Unnamed: 0,bathrooms,bedrooms,city,country,currency,daysOnZillow,homeStatus,homeStatusForHDP,homeType,isFeatured,isNonOwnerOccupied,isPreforeclosureAuction,isPremierBuilder,isRentalWithBasePrice,isUnmappable,isZillowOwned,latitude,area,longitude,rent,priceForHDP,shouldHighlight,state,street_address,unit,zipcode,zpid,lotAreaUnit,lotAreaValue,rentZestimate,taxAssessedValue,zestimate,datePriceChanged,priceChange,priceReduction
0,0,1.0,1.0,Springfield,USA,USD,-1,FOR_RENT,FOR_RENT,APARTMENT,True,True,False,False,False,False,False,42.104,617.0,-72.592384,1042.0,1042.0,False,MA,193 Worthington St #71DFDFD55,# 71dfdfd55,01103,2066685043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## List of Springfield, MA neighborhoods with zipcodes ##
### May be able to make this into a dictionary and use 'clean swiss data' function in notebook 2 to assign neighborhood names to zip codes ###

### *combined a few of the zip codes that had mulitple neighborhoods associated with them ###
 
    Atwater Park, 01107
    Downtown, 01103, 01105
    East Forest Park, 01118
    Forest Park, 01108
    Indian Orchard, 01151
    Liberty Heights & East Springfield, 01104
    Maple Hill/Ridgewood, 01105
    McKnight, 01109
    Pine Point/Boston Road, 01109, 01119
    Sixteen Acres, 01118, 01119, 01128, 01129


In [7]:
hood_dict = {'01107':'Atwater Park',
             '01103':'Downtown',
             '01118':'East Forest Park*',
             '01108':'Forest Park',
             '01151':'Indian Orchard',
             '01104':'Liberty Heights & East Springfield;',
             '01105':'Maple Hill/Ridgewood*',
             '01109':'McKnight*',
             '01119':'Pine Point/Boston Road*',
             '01128':'Sixteen Acres',
             '01129':'SixteenAcres',
            }

In [8]:
def create_new_features(df):
    # Add rent categories
    rent_all_columns['sqft_cost'] =  rent_all_columns["rent"] /  rent_all_columns["area"]
    descr_rents =  rent_all_columns['sqft_cost'].describe()

    quantiles =  rent_all_columns['sqft_cost'].quantile(q=[0.15, 0.5, 0.85])
    cheap = quantiles[0.15]
    average = quantiles[0.5]
    expensive = quantiles[0.85]

    rent_all_columns["cost_per_square_feet"] = np.where(
         rent_all_columns['sqft_cost'] < cheap,
        0,
        np.where(
            ( rent_all_columns['sqft_cost'] >= cheap) & ( rent_all_columns['sqft_cost'] < average),
            1,
            np.where(
                ( rent_all_columns['sqft_cost'] >= average) & ( rent_all_columns['sqft_cost'] < expensive),
                2,
                3,
            ),
        ),
    )
    rent_all_columns["hover_strings_scatter"] = [
    f"Address: {street}, {place},<br>Rooms: {rooms}, <br>Rent: USD {rent}"
    for street, place, rooms, rent in zip(
        rent_all_columns["street_address"],
        rent_all_columns["city"],
        rent_all_columns["bedrooms"],
        rent_all_columns["rent"],
        )
    ]
    return  rent_all_columns
rent_all_columns.head(1)

,Unnamed: 0,bathrooms,bedrooms,city,country,currency,daysOnZillow,homeStatus,homeStatusForHDP,homeType,isFeatured,isNonOwnerOccupied,isPreforeclosureAuction,isPremierBuilder,isRentalWithBasePrice,isUnmappable,isZillowOwned,latitude,area,longitude,rent,priceForHDP,shouldHighlight,state,street_address,unit,zipcode,zpid,lotAreaUnit,lotAreaValue,rentZestimate,taxAssessedValue,zestimate,datePriceChanged,priceChange,priceReduction
0,0,1.0,1.0,Springfield,USA,USD,-1,FOR_RENT,FOR_RENT,APARTMENT,True,True,False,False,False,False,False,42.104,617.0,-72.592384,1042.0,1042.0,False,MA,193 Worthington St #71DFDFD55,# 71dfdfd55,01103,2066685043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Adding 'sqft_cost' column ##

In [ ]:
rent_all_columns['sqft_cost'] =  rent_all_columns["rent"] /  rent_all_columns["area"]
descr_rents =  rent_all_columns['sqft_cost'].describe()
rent_all_columns.head(1)

## Adding 'hover_strings_scatter' column ##

In [ ]:
rent_all_columns["hover_strings_scatter"] = [
    f"Address: {street}, {place},<br>Rooms: {rooms}, <br>Rent: USD {rent}"
    for street, place, rooms, rent in zip(
        rent_all_columns["street_address"],
        rent_all_columns["city"],
        rent_all_columns["bedrooms"],
        rent_all_columns["rent"],
        )
    ]
rent_all_columns.head(1)

## Adding 'quantile calculations' ##

In [ ]:
quantiles =  rent_all_columns['sqft_cost'].quantile(q=[0.15, 0.5, 0.85])
cheap = quantiles[0.15]
average = quantiles[0.5]
expensive = quantiles[0.85]

rent_all_columns["cost_per_square_feet"] = np.where(rent_all_columns['sqft_cost'] < cheap, 0,
        np.where(( rent_all_columns['sqft_cost'] >= cheap) & ( rent_all_columns['sqft_cost'] < average), 1,
        np.where(( rent_all_columns['sqft_cost'] >= average) & ( rent_all_columns['sqft_cost'] < expensive), 2, 3,
            ),
        ),
    )
rent_all_columns.head(1)

## Getting only wanted columns ##

In [ ]:
rent_data = rent_all_columns[['bedrooms', 'bathrooms', 'area', 'rent', 'street_address', 'zipcode', 'city', 'sqft_cost','hover_strings_scatter','cost_per_square_feet']]
rent_data.head(1)

### need to write a fucntion that appends neighborhood names to rows with matching zipcodes and create a new columns called 'neighborhoods' ###

### Adding a hover string column to the data frame ###

## *Need to add remaing 'create new features' function from notebook 2* ##

def create_new_features(df):
    # Add rent categories
    rent_all_columns['sqft_cost'] =  rent_all_columns["rent"] /  rent_all_columns["area"]
    descr_rents =  rent_all_columns['sqft_cost'].describe()

    quantiles =  rent_all_columns['sqft_cost'].quantile(q=[0.15, 0.5, 0.85])
    cheap = quantiles[0.15]
    average = quantiles[0.5]
    expensive = quantiles[0.85]

    rent_all_columns["cost_per_square_feet"] = np.where(
         rent_all_columns['sqft_cost'] < cheap,
        0,
        np.where(
            ( rent_all_columns['sqft_cost'] >= cheap) & ( rent_all_columns['sqft_cost'] < average),
            1,
            np.where(
                ( rent_all_columns['sqft_cost'] >= average) & ( rent_all_columns['sqft_cost'] < expensive),
                2,
                3,
            ),
        ),
    )
    rent_all_columns["hover_strings_scatter"] = [
    f"Address: {street}, {place},<br>Rooms: {rooms}, <br>Rent: USD {rent}"
    for street, place, rooms, rent in zip(
        rent_all_columns["street_address"],
        rent_all_columns["city"],
        rent_all_columns["bedrooms"],
        rent_all_columns["rent"],
        )
    ]
    return  rent_all_columns

## ***Need to re-add 'area' to function once NaN values are taken care of*** ##

In [ ]:
 # Hover strings
#rent_all_columns["hover_strings_scatter"] = [
#    f"Address: {street}, {place},<br>Rooms: {rooms}, Floor Space: {round(size)}m²,<br>Rent: USD {rent}"
#    for street, place, rooms, size, rent in zip(
#        rent_all_columns["street_address"],
#        rent_all_columns["city"],
#        rent_all_columns["bedrooms"],
#        rent_all_columns["area"],
#        rent_all_columns["rent"],
#    )
#]

rent_all_columns["hover_strings_scatter"] = [
    f"Address: {street}, {place},<br>Rooms: {rooms}, <br>Rent: USD {rent}"
    for street, place, rooms, rent in zip(
        rent_all_columns["street_address"],
        rent_all_columns["city"],
        rent_all_columns["bedrooms"],
        rent_all_columns["rent"],
    )
]

# Rent Data all clean and making csv file and getting other information #

In [ ]:
rent_data.head(2)

In [ ]:
rent_data.to_csv('rent_data_clean_spfld.csv', encoding='utf-8')

In [ ]:
rent_data.info()

In [ ]:
rent_data.describe()

In [ ]:
rent_data.value_counts()